Goal of this notebook:

Perform feature selection on our dataset.

Strategy:

Iterate over each project and execute the feature selection

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore")

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=100, max_features=0.3, min_samples_leaf=1)

# Tree-based feature selection

Uses the feature_importances_ attribute from the Random Forest model to select the most important features. It uses the mean of the importances of all features as a threshold.

In [5]:
import importlib
importlib.reload(classifier_utils)
results_tree, attributes_record_tree = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'tree')

In [6]:
results_tree

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,33.0,0.740,0.734,-0.009
1,apache__directory-server,652.0,96.00,19.0,0.939,0.925,-0.015
2,android__platform_frameworks_base,2460.0,566.00,54.0,0.818,0.815,-0.004
3,freenet__fred,1012.0,134.00,47.0,0.679,0.680,0.003
4,alexo__wro4j,1368.0,107.00,30.0,0.586,0.577,-0.015
5,apache__lucene-solr,974.0,124.00,42.0,0.650,0.639,-0.017
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,30.0,0.710,0.710,-0.000
8,atlasapi__atlas,782.0,124.00,42.0,0.662,0.652,-0.015
9,hibernate__hibernate-orm,716.0,131.00,35.0,0.598,0.595,-0.005


# Recursive feature elimination:

First, the estimator is trained on the initial set of features and the importance of each feature is obtained. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
Currently we are discarding 1 feature per step, using a 5-fold-cross-validation to calculate the accuracy on each step.

In [7]:
import importlib
importlib.reload(classifier_utils)
results_recursive, attributes_record_rec = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'recursive')

In [8]:
results_recursive

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,120.00,0.740,0.748,0.030
1,apache__directory-server,652.0,96.00,41.00,0.939,0.937,-0.002
2,android__platform_frameworks_base,2460.0,566.00,138.00,0.818,0.816,-0.002
3,freenet__fred,1012.0,134.00,37.00,0.679,0.685,0.018
4,alexo__wro4j,1368.0,107.00,101.00,0.586,0.593,0.018
5,apache__lucene-solr,974.0,124.00,107.00,0.650,0.651,0.003
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,67.00,0.710,0.701,-0.012
8,atlasapi__atlas,782.0,124.00,114.00,0.662,0.652,-0.015
9,hibernate__hibernate-orm,716.0,131.00,127.00,0.598,0.592,-0.009


# IGAR
Selects attributes based on the ranking of their information gain.
Information gain measures the ability of a feature to separate the target classes. The greater the information gain, the better its importance for classification tasks.

Information Gain = Entropy(overall) - Entropy(attribute)

The algorithm has an input value 'n' that is used to select the 'n' attributes with the greatest information gain among all attributes. In this notebook we use n = 81, which was the found in the notebook IGAR_tuning.ipynb.

In [9]:
import importlib
importlib.reload(classifier_utils)
results_IGAR, attributes_record_IGAR = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'IGAR')

In [10]:
results_IGAR

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,81.0,0.740,0.740,-0.000
1,apache__directory-server,652.0,96.00,81.0,0.939,0.942,0.050
2,android__platform_frameworks_base,2460.0,566.00,81.0,0.818,0.812,-0.008
3,freenet__fred,1012.0,134.00,81.0,0.679,0.671,-0.012
4,alexo__wro4j,1368.0,107.00,81.0,0.586,0.580,-0.010
5,apache__lucene-solr,974.0,124.00,81.0,0.650,0.636,-0.022
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,81.0,0.710,0.713,0.012
8,atlasapi__atlas,782.0,124.00,81.0,0.662,0.657,-0.008
9,hibernate__hibernate-orm,716.0,131.00,81.0,0.598,0.592,-0.009


## Comparison

In [11]:
df_inner = pd.merge(results_tree, results_recursive, on='project', how='inner', suffixes=('_tree', '_rec'))
df_inner_igar = results_IGAR.add_suffix("_IGAR").rename(columns={"project_IGAR": "project"})
df_inner = pd.merge(df_inner, df_inner_igar, on='project', how='inner')
df_inner.to_csv('feature_selection_comparison.csv', index=False)

accuracy_inner = df_inner.filter(regex=("project|accuracy.*")).copy()
accuracy_inner['improvement_tree'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_tree'], x['accuracy_tree']), axis=1)
accuracy_inner['improvement_rec'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_rec'], x['accuracy_rec']), axis=1)
accuracy_inner['improvement_IGAR'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_IGAR'], x['accuracy_IGAR']), axis=1)
accuracy_inner = accuracy_inner.round(3)
accuracy_inner

,project,accuracy_tree,accuracy_fs_tree,accuracy_rec,accuracy_fs_rec,accuracy_IGAR,accuracy_fs_IGAR,improvement_tree,improvement_rec,improvement_IGAR
0,Ramblurr__Anki-Android,0.740,0.734,0.740,0.748,0.740,0.740,-0.008,0.031,0.000
1,apache__directory-server,0.939,0.925,0.939,0.937,0.939,0.942,-0.015,-0.002,0.049
2,android__platform_frameworks_base,0.818,0.815,0.818,0.816,0.818,0.812,-0.004,-0.002,-0.007
3,freenet__fred,0.679,0.680,0.679,0.685,0.679,0.671,0.003,0.019,-0.012
4,alexo__wro4j,0.586,0.577,0.586,0.593,0.586,0.580,-0.015,0.017,-0.010
5,apache__lucene-solr,0.650,0.639,0.650,0.651,0.650,0.636,-0.017,0.003,-0.022
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,0.710,0.710,0.710,0.701,0.710,0.713,0.000,-0.013,0.010
8,atlasapi__atlas,0.662,0.652,0.662,0.652,0.662,0.657,-0.015,-0.015,-0.008
9,hibernate__hibernate-orm,0.598,0.595,0.598,0.592,0.598,0.592,-0.005,-0.010,-0.010


In [12]:
pd.DataFrame(attributes_record_rec, columns=['project', 'attribute', 'information_gain', 'method']).to_csv('attributes_record_rec.csv', index=False)

In [13]:
attributes_record = []
attributes_record.extend(attributes_record_tree)
attributes_record.extend(attributes_record_rec)
attributes_record.extend(attributes_record_IGAR)
attributes_record_df = pd.DataFrame(attributes_record, columns=['project', 'attribute', 'information_gain', 'method'])
attributes_record_df.to_csv('attributes_record.csv', index=False)

## Ranking of features selected by tree method

Counts in how many projects the respective feature was selected using the tree method.

In [14]:
ranking_tree = classifier_utils.get_attribute_selection_ranking(attributes_record, 'tree')
ranking_tree.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
69,steve@hibernate.org,1.0,2.069798,12.000000
10,keyword_remove,17.0,1.721044,7.058824
26,Merge isolation time,22.0,1.670107,2.727273
25,Branching time,20.0,1.667334,2.250000
29,Changed files 1,20.0,1.648005,2.950000
27,Commits 1,20.0,1.643616,4.150000
8,keyword_update,16.0,1.630010,8.187500
11,keyword_use,18.0,1.627590,6.000000
1,left_lines_removed,20.0,1.613181,5.300000
6,keyword_bug,18.0,1.610705,7.277778


## Ranking of features selected by recursive method

Counts in how many projects the respective feature was selected using the recursive method.

In [15]:
ranking_recursive = classifier_utils.get_attribute_selection_ranking(attributes_record, 'recursive')
ranking_recursive.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
205,alex.objelean@gmail.com,1.0,2.091704,6.000000
336,steve@hibernate.org,1.0,2.069798,14.000000
12,keyword_add,20.0,1.681671,5.800000
29,Merge isolation time,22.0,1.670107,3.000000
28,Branching time,22.0,1.669547,3.000000
3,right_lines_removed,22.0,1.645285,1.772727
2,right_lines_added,22.0,1.640534,2.681818
32,Different devs,21.0,1.635892,7.333333
35,Commits 1,22.0,1.630672,4.681818
5,keyword_fix,22.0,1.622875,5.727273


## Ranking of features selected by IGAR method

Counts in how many projects the respective feature was selected using the IGAR method. 

The information gain column is an average among all projects.

In [16]:
import importlib
importlib.reload(classifier_utils)
ranking_IGAR = classifier_utils.get_attribute_selection_ranking(attributes_record, 'IGAR')
ranking_IGAR.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
106,steve@hibernate.org,1.0,2.069798,14.00
105,michael@getrailo.org,1.0,1.588456,18.00
37,fileSize,25.0,1.509932,1.64
0,chunkRelSize,25.0,1.509770,1.52
3,Merge isolation time,25.0,1.503189,3.12
5,Branching time,25.0,1.502696,3.36
19,Changed files 1,25.0,1.498969,4.36
39,fileCC,25.0,1.496872,3.76
22,Commits 1,25.0,1.496374,5.12
16,keyword_add,25.0,1.485307,5.88
